In [ ]:
# Need to install additional packages for the notebook
! pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 8.8 MB/s  0:00:01eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 10.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 10.1 MB/s  0:00:00eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [streamlit]/8 [streamlit]


# Streamlit Protyping

## Imports

In [4]:
import streamlit as st
import os
import pandas as pd, numpy as np
import oandapyV20
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.instruments as instruments

## Parameters, Classes, and Functions

In [5]:
OANDA_ACCOUNT_ID = os.getenv("OANDA_ACCOUNT_ID")
OANDA_TOKEN = os.getenv("OANDA_TOKEN")

def get_latest_price(account_id, access_token, instrument="SPX500_USD"):
    client = oandapyV20.API(access_token=access_token)
    params = {"instruments": instrument}
    r = pricing.PricingInfo(accountID=account_id, params=params)
    client.request(r)
    prices = r.response['prices'][0]
    bid = float(prices['bids'][0]['price'])
    ask = float(prices['asks'][0]['price'])
    return (bid + ask) / 2

def get_candles(account_id, access_token, instrument="SPX500_USD", granularity="H1", count=100):
    client = oandapyV20.API(access_token=access_token)
    params = {"granularity": granularity, "count": count}
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    client.request(r)
    return r.response['candles']

## App

In [ ]:
st.set_page_config(page_title="S&P500 Volatility Dashboard", layout="wide")
st.title("S&P500 Volatility Dashboard")

# ---- SIDEBAR ----
st.sidebar.header("Settings")
instrument = st.sidebar.text_input("Instrument", "SPX500_USD")
granularity = st.sidebar.selectbox("Granularity", ["M15", "H1", "D"], index=1)
refresh = st.sidebar.slider("Auto-refresh (seconds)", 10, 300, 60)

# ---- FETCH OANDA DATA ----
st.header(f"OANDA Live Data: {instrument}")
price = get_latest_price(OANDA_ACCOUNT_ID, OANDA_TOKEN, instrument)
st.metric("Current Price", f"{price:.2f} USD")

candles = get_candles(OANDA_ACCOUNT_ID, OANDA_TOKEN, instrument, granularity)
df = pd.DataFrame([
    {
        "time": c["time"],
        "open": float(c["mid"]["o"]),
        "high": float(c["mid"]["h"]),
        "low": float(c["mid"]["l"]),
        "close": float(c["mid"]["c"]),
    } for c in candles
])

# ---- CHART ----
fig = go.Figure(data=[go.Candlestick(
    x=df["time"],
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"]
)])
st.plotly_chart(fig, use_container_width=True)

# ---- ML INFERENCE ----
# st.subheader("Predicted Volatility (Model Output)")
# features = df[["open", "high", "low", "close"]].tail(1)
# vol_pred = predict_volatility(features)
# st.metric("Predicted Volatility", f"{vol_pred[0]:.4f}")